# We ballin out here

imports

In [210]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

## Finding Player

In [238]:
from nba_api.stats.static import players
name = "Kevin Durant"
player = list(filter(lambda x: x["is_active"], players.find_players_by_full_name(name)))
print(player)
player_id = player[0]['id']

[{'id': 201142, 'full_name': 'Kevin Durant', 'first_name': 'Kevin', 'last_name': 'Durant', 'is_active': True}]
None


## Find Career Stats and current team

In [250]:
from nba_api.stats.endpoints import playercareerstats
career = playercareerstats.PlayerCareerStats(player_id = player_id).get_data_frames()
team = list(career[0]["TEAM_ID"])[-1]
career[0]

1610612751


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,201142,2007-08,00,1610612760,SEA,19.0,80,80,2768.0,587,...,0.873,70,278,348,192,78,75,232,122,1624
1,201142,2008-09,00,1610612760,OKC,20.0,74,74,2885.0,661,...,0.863,77,405,482,205,96,53,225,134,1871
2,201142,2009-10,00,1610612760,OKC,21.0,82,82,3239.0,794,...,0.900,105,518,623,231,112,84,271,171,2472
3,201142,2010-11,00,1610612760,OKC,22.0,78,78,3038.0,711,...,0.880,57,476,533,214,88,76,218,159,2161
4,201142,2011-12,00,1610612760,OKC,23.0,66,66,2546.0,643,...,0.860,40,487,527,231,88,77,248,133,1850
5,201142,2012-13,00,1610612760,OKC,24.0,81,81,3119.0,731,...,0.905,46,594,640,374,116,105,280,143,2280
6,201142,2013-14,00,1610612760,OKC,25.0,81,81,3122.0,849,...,0.873,58,540,598,445,103,59,285,174,2593
7,201142,2014-15,00,1610612760,OKC,26.0,27,27,913.0,238,...,0.854,16,162,178,110,24,25,74,40,686
8,201142,2015-16,00,1610612760,OKC,27.0,72,72,2578.0,698,...,0.898,45,544,589,361,69,85,250,137,2029
9,201142,2016-17,00,1610612744,GSW,28.0,62,62,2070.0,551,...,0.875,39,474,513,300,66,99,138,117,1555


## Find Player's Games Next Week

In [221]:
# https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/playernextngames.md 
from nba_api.stats.endpoints import playernextngames
number_games = 10
next_games = playernextngames.PlayerNextNGames(number_of_games = number_games, player_id = player_id)
next_games = next_games.get_data_frames()[0]

# convert dates to Date objects
date_format = "%b %d, %Y"
next_games['GAME_DATE'] = next_games['GAME_DATE'].map(lambda d: datetime.strptime(d, date_format).date())
next_games

,GAME_ID,GAME_DATE,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_NAME,VISITOR_TEAM_NAME,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_NICKNAME,GAME_TIME,HOME_WL,VISITOR_WL
0,0022000472,2021-02-21,1610612746,1610612751,LA Clippers,Brooklyn,LAC,BKN,Clippers,Nets,08:00 PM,22-9,19-12
1,0022000482,2021-02-23,1610612751,1610612758,Brooklyn,Sacramento,BKN,SAC,Nets,Kings,07:30 PM,19-12,12-16
2,0022000499,2021-02-25,1610612751,1610612753,Brooklyn,Orlando,BKN,ORL,Nets,Magic,07:30 PM,19-12,12-18
3,0022000519,2021-02-27,1610612751,1610612742,Brooklyn,Dallas,BKN,DAL,Nets,Mavericks,08:30 PM,19-12,13-15
4,0022000534,2021-03-01,1610612759,1610612751,San Antonio,Brooklyn,SAS,BKN,Spurs,Nets,08:30 PM,16-11,19-12
5,0022000546,2021-03-03,1610612745,1610612751,Houston,Brooklyn,HOU,BKN,Rockets,Nets,07:30 PM,11-17,19-12


In [216]:
# Get Next Monday
week = 1
this_Monday = datetime.now().date() - timedelta(days=datetime.now().date().weekday()) + timedelta(weeks=week)
next_Monday = datetime.now().date() - timedelta(days=datetime.now().date().weekday()) + timedelta(weeks=week+1)
print(this_Monday, next_Monday)

2021-02-22 2021-03-01


In [251]:
# Find games of the week
after_monday = next_games[next_games["GAME_DATE"] >= this_Monday]
games_of_week = after_monday[after_monday["GAME_DATE"] < next_Monday]
games_of_week

,GAME_ID,GAME_DATE,HOME_TEAM_ID,VISITOR_TEAM_ID,HOME_TEAM_NAME,VISITOR_TEAM_NAME,HOME_TEAM_ABBREVIATION,VISITOR_TEAM_ABBREVIATION,HOME_TEAM_NICKNAME,VISITOR_TEAM_NICKNAME,GAME_TIME,HOME_WL,VISITOR_WL
1,0022000482,2021-02-23,1610612751,1610612758,Brooklyn,Sacramento,BKN,SAC,Nets,Kings,07:30 PM,19-12,12-16
2,0022000499,2021-02-25,1610612751,1610612753,Brooklyn,Orlando,BKN,ORL,Nets,Magic,07:30 PM,19-12,12-18
3,0022000519,2021-02-27,1610612751,1610612742,Brooklyn,Dallas,BKN,DAL,Nets,Mavericks,08:30 PM,19-12,13-15


## Find Fantasy Stats for the Season

https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/playerfantasyprofile.md

In [252]:
from nba_api.stats.endpoints import playerfantasyprofile
fantasy_profile = playerfantasyprofile.PlayerFantasyProfile(player_id = player_id).get_data_frames()
fantasy_profile[4]

,GROUP_SET,GROUP_VALUE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,DD2,TD3,FAN_DUEL_PTS,NBA_FANTASY_PTS
0,vs. Opponent,Atlanta Hawks,3,2,1,0.667,114.586667,34,69,0.493,...,5,5,3,15,93,-2,1,0,148.3,155.3
1,vs. Opponent,Boston Celtics,1,1,0,1.000,33.400000,9,16,0.563,...,0,2,3,6,29,31,0,0,40.3,42.3
2,vs. Opponent,Charlotte Hornets,1,0,1,0.000,36.350000,9,15,0.600,...,1,0,3,4,29,7,0,0,37.6,38.6
3,vs. Opponent,Cleveland Cavaliers,1,0,1,0.000,50.150000,12,25,0.480,...,4,1,2,9,38,2,1,0,72.4,77.4
4,vs. Opponent,Denver Nuggets,1,1,0,1.000,35.880000,12,18,0.667,...,0,1,4,5,34,15,1,0,57.3,57.3
5,vs. Opponent,Golden State Warriors,2,2,0,1.000,58.213333,15,35,0.429,...,3,4,3,9,42,48,0,0,74.5,80.5
6,vs. Opponent,LA Clippers,1,1,0,1.000,38.166667,11,13,0.846,...,1,0,3,4,28,-8,0,0,35.8,36.8
7,vs. Opponent,Miami Heat,2,2,0,1.000,74.713333,17,40,0.425,...,4,1,5,11,51,17,1,0,89.9,94.9
8,vs. Opponent,Milwaukee Bucks,1,1,0,1.000,35.706667,10,21,0.476,...,2,0,5,6,30,0,0,0,50.8,52.8
9,vs. Opponent,New York Knicks,1,1,0,1.000,30.350000,10,18,0.556,...,2,0,1,4,26,12,0,0,41.6,43.6


### Find player performance into next teams